<a href="https://colab.research.google.com/github/vamosya/data_notebooks_python/blob/main/GarminActivitiesToMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
pip install garminconnect

In [24]:
pip install reverse_geocoder

In [25]:
# Librerias necesarias
from garminconnect import Garmin
import logging
from getpass import getpass
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [26]:
# Conexión a Garmin Connect
paswd = getpass()
client = Garmin("cperaltap@gmail.com", paswd)
client.login()
print("Hola {}! Estás dentro!!".format(client.get_full_name()))

··········
Hola Txiki! Estás dentro!!


In [28]:
# Bajar actividades
activities = client.get_activities(0,2000) # tope de 2000. Se puede modificar

# guardo en un dataframe de Pandas
activities = pd.DataFrame(activities)

# Muestro id de la primera actividad
activities['activityId'][0]

7707003773

In [30]:
# Extraer de ActivityType el deporte y añadirlo a columna
sports = []
for i in range(len(activities)):
  sports.append(activities['activityType'][i]['typeKey'])
activities['sport'] = sports

# Muestro los deportes encontrados
activities['sport'].unique()

array(['running', 'hiking', 'mountain_biking', 'cycling', 'paddling',
       'stand_up_paddleboarding', 'open_water_swimming', 'inline_skating',
       'road_biking', 'rowing', 'strength_training', 'indoor_cycling',
       'treadmill_running', 'other', 'indoor_cardio', 'lap_swimming',
       'skating_ws', 'swimming', 'walking', 'boating', 'virtual_run',
       'elliptical', 'recumbent_cycling', 'rock_climbing',
       'indoor_rowing', 'fitness_equipment', 'tennis', 'casual_walking'],
      dtype=object)

In [39]:
# Paso a str y gestiono los "nan"
activities['startLatitude'] = activities['startLatitude'].astype(str)
activities['startLongitude'] = activities['startLongitude'].astype(str)
activities['startLatitude'].replace({"nan": "0.0"}, inplace=True)
activities['startLongitude'].replace({"nan": "0.0"}, inplace=True)

# Ignoro las coordenadas 0
activities = activities[activities['startLatitude'] != 0 ]

In [44]:
# Cargar folium y crear data
import folium

# Paso las coordenadas a decimales
activities['startLatitude'] = activities['startLatitude'].astype(float)
activities['startLongitude'] = activities['startLongitude'].astype(float)

# Defino el mapa y su localización central
mapa = folium.Map( location=[activities['startLatitude'][0], activities['startLongitude'][0]] )

# Itero para elegir los colores por deporte y para ir creando los puntos
for i in range(len(activities)):
  if activities['sport'][i] == 'running':
    color = 'green'
  elif activities['sport'][i] == 'mountain_biking':
    color = 'blue'
  elif activities['sport'][i] == 'cycling':
    color = 'blue'
  else:
    color = 'red'
  folium.Marker([activities['startLatitude'][i], activities['startLongitude'][i]], icon=folium.Icon(color= color), tooltip=activities['activityName'][i]).add_to(mapa)

In [45]:
mapa